In [ ]:
! unzip '/content/credit_card.zip'

Archive:  /content/credit_card.zip
  inflating: creditcard.csv          


In [ ]:
! pip install deap

#02

In [ ]:
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from deap import base, creator, tools, algorithms

In [ ]:
# Create the creator objects outside of any loops or function calls
creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

# Load and preprocess data
data = pd.read_csv("/content/creditcard.csv")
data
# Preprocess data as needed...
X = data.drop("Class", axis=1)
y = data["Class"]

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from deap import base, creator, tools, algorithms
import random

toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 10, 100)  # Number of trees
toolbox.register("attr_int", random.randint, 1, 10)    # Max depth
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_int, toolbox.attr_int), n=1)

def evaluate(individual):
    num_trees, max_depth = individual
    clf = RandomForestClassifier(n_estimators=num_trees + 1, max_depth=max_depth+1, random_state=42)
    # clf = RandomForestClassifier(n_estimators=max(num_trees, 1), max_depth=max(max_depth,1), random_state=42)
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    complexity = -max_depth  # Minimize negative depth as complexity
    return accuracy, complexity

def moead_selection(population, toolbox, num_selected):
    selected = []

    for num in range(num_selected):
        ind1, ind2 = random.sample(population, 2)

        # Compared individuals based on their fitness values
        if ind1.fitness < ind2.fitness:
            selected.append(ind1)
        else:
            selected.append(ind2)

    return selected

# Registerd the evaluation and selection functions
toolbox.register("evaluate", evaluate)
toolbox.register("select", moead_selection)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
population_size = 50
generations = 50
num_selected = 10  # Number of individuals selected in each generation

population = [toolbox.individual() for i in range(population_size)]

# Initialize fitness values for the initial population
for ind in population:
    ind.fitness.values = toolbox.evaluate(ind)

# MOEA/D algorithm
for gen in range(generations):
    offspring = []

    # Generated offspring through variation operators
    for _ in range(population_size):
        ind1, ind2 = random.sample(population, 2)
        child = toolbox.clone(ind1)
        toolbox.mate(child, ind2)
        toolbox.mutate(child)
        offspring.append(child)

    # Evaluation of the offspring
    for ind in offspring:
        ind.fitness.values = toolbox.evaluate(ind)

    # Selected individuals for the next generation
    population = toolbox.select(population + offspring, toolbox, population_size)

# Identified the best individual from the final population
best_individual = min(population, key=lambda ind: ind.fitness.values)

# Extracted the best hyperparameters and evaluate the final model
best_num_trees, best_max_depth = best_individual
final_clf = RandomForestClassifier(n_estimators=best_num_trees, max_depth=best_max_depth, random_state=42)
final_clf.fit(X_train_scaled, y_train)
final_y_pred = final_clf.predict(X_test_scaled)
final_accuracy = accuracy_score(y_test, final_y_pred)

print(f"Final Model Accuracy: {final_accuracy}")


In [ ]:
# Define the optimization problem
toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 10, 100)  # Number of trees
toolbox.register("attr_int", random.randint, 1, 10)    # Max depth
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_int, toolbox.attr_int), n=1)

def evaluate(individual):
    num_trees, max_depth = individual
    clf = RandomForestClassifier(n_estimators=num_trees + 1, max_depth=max_depth+1, random_state=42)
    # clf = RandomForestClassifier(n_estimators=max(num_trees, 1), max_depth=max(max_depth,1), random_state=42)
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    complexity = -max_depth  # Minimize negative depth as complexity
    return accuracy, complexity


toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selNSGA2)

# Implement MOEA/D algorithm
population_size = 50
generations = 50
cx_prob = 0.7
mut_prob = 0.2

population = [toolbox.individual() for _ in range(population_size)]

fitnesses = map(toolbox.evaluate, population)
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit

algorithms.eaMuPlusLambda(population, toolbox, mu=population_size, lambda_=population_size,
                          cxpb=cx_prob, mutpb=mut_prob, ngen=generations,
                          stats=None, halloffame=None, verbose=True)

pareto_front = tools.sortNondominated(population, len(population), first_front_only=True)[0]

best_hyperparameters = pareto_front[0]
best_num_trees, best_max_depth = best_hyperparameters

final_clf = RandomForestClassifier(n_estimators=best_num_trees, max_depth=best_max_depth, random_state=42)

# final_clf = RandomForestClassifier()
final_clf.fit(X_train_scaled, y_train)
final_y_pred = final_clf.predict(X_test_scaled)
final_accuracy = accuracy_score(y_test, final_y_pred)

print(f"Final Model Accuracy: {final_accuracy}")


gen	nevals
0  	0     
1  	5     
2  	4     
3  	5     
4  	5     
5  	3     
Final Model Accuracy: 0.9995435553526912


#O1

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score

In [ ]:
# # Load the credit card dataset (assuming it's in a CSV file)
# data = pd.read_csv('/content/creditcard.csv')
# data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
# Preprocess the data
X = data.drop('Class', axis=1)
y = data['Class']

In [ ]:
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# def initialize_population(pop_size):
#     population = []
#     for _ in range(pop_size):
#         # Initialize Random Forest hyperparameters
#         rf_params = {
#             'n_estimators': np.random.randint(50, 200),
#             'max_depth': np.random.randint(5, 20),
#             # Add other hyperparameters
#         }

#         # Initialize fitness values (accuracy and diversity)
#         individual = {
#             'params': rf_params,
#             'fitness': {'accuracy': 0.0, 'diversity': 0.0}
#         }

#         population.append(individual)

#     return str(population)



In [ ]:
# def crossover_and_mutation(parent1, parent2):
#     child_params = {}

#     for param in parent1['params']:
#         if np.random.rand() < 0.5:
#             child_params[param] = parent1['params'][param]
#         else:
#             child_params[param] = parent2['params'][param]

#     # Apply mutation with a small probability
#     for param in child_params:
#         if np.random.rand() < 0.1:
#             mutation_amount = np.random.randint(-10, 10)
#             child_params[param] += mutation_amount

#     child = {'params': child_params, 'fitness': {'accuracy': 0.0, 'diversity': 0.0}}
#     return child






In [ ]:
# def select_parents(subproblem, population):
#     tournament_size = 2
#     candidates = np.random.choice(len(population), size=tournament_size, replace=False)

#     best_parent = None
#     best_fitness = None
#     for candidate_index in candidates:
#         candidate = population[candidate_index]
#         candidate_fitness = calculate_scalar_fitness(candidate, subproblem)
#         if best_parent is None or dominates(candidate_fitness, best_fitness):
#             best_parent = candidate
#             best_fitness = candidate_fitness

#     return best_parent

# def update_population(population, new_individuals):
#     for i in range(len(population)):
#         population[i] = new_individuals[i]



In [ ]:
# def dominates(fitness1, fitness2):
#     if fitness1['accuracy'] >= fitness2['accuracy'] and fitness1['diversity'] <= fitness2['diversity']:
#         return True
#     return False

# def calculate_scalar_fitness(individual, subproblem):
#     accuracy_weight = subproblem[0]
#     diversity_weight = subproblem[1]

#     accuracy = individual['fitness']['accuracy']
#     diversity = individual['fitness']['diversity']

#     scalar_fitness = accuracy_weight * accuracy + diversity_weight * diversity

#     return str(scalar_fitness)

# def evaluate_fitness(params, X_train, y_train, X_valid, y_valid):
#     model = RandomForestClassifier(**params)
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_valid)

#     accuracy = accuracy_score(y_valid, y_pred)
#     diversity = calculate_diversity(model)

#     return str({'accuracy': accuracy, 'diversity': diversity})

# def calculate_diversity(model):
#     trees = model.estimators_
#     num_trees = len(trees)

#     diversity = 0.0
#     for i in range(num_trees):
#         for j in range(i + 1, num_trees):
#             tree_difference = calculate_tree_difference(trees[i], trees[j])
#             diversity += tree_difference

#     normalized_diversity = diversity / (num_trees * (num_trees - 1) / 2)

#     return normalized_diversity

# def calculate_tree_difference(tree1, tree2):
#     importance1 = tree1.feature_importances_
#     importance2 = tree2.feature_importances_

#     tree_difference = np.sum(np.abs(importance1 - importance2))

#     return tree_difference

In [ ]:
# def generate_weights(pop_size):
#     weights = []
#     for _ in range(pop_size):
#         w1 = np.random.uniform(0, 1)
#         w2 = 1 - w1
#         weights.append([w1, w2])
#     return weights

# def extract_pareto_front(population):
#     pareto_front = []
#     for individual in population:
#         if not any(dominates(individual['fitness'], other['fitness']) for other in population):
#             pareto_front.append(individual)
#     return pareto_front

# def decompose_objectives(population, weights):
#     subproblems = []
#     for i, individual in enumerate(population):
#         if i < len(weights):
#             subproblem = weights[i]
#         else:
#             subproblem = weights[-1]
#         subproblems.append(subproblem)
#     return subproblems

In [ ]:
# def moead_random_forest(X_train, y_train, X_valid, y_valid, pop_size, max_generations):
#     population = initialize_population(pop_size)

#     for generation in range(max_generations):
#         weights = generate_weights(pop_size)
#         subproblems = decompose_objectives(population, weights)

#         new_individuals = []
#         for i, subproblem in enumerate(subproblems):
#             sparent1, parent2 = select_parents(subproblem, population)
#             child = crossover_and_mutation(parent1, parent2)
#             child_fitness = evaluate_fitness(child['params'], X_train, y_train, X_valid, y_valid)

#             child['fitness']['accuracy'] = child_fitness['accuracy']
#             child['fitness']['diversity'] = child_fitness['diversity']

#             new_individuals.append(child)

#         update_population(population, new_individuals)

#     pareto_front = extract_pareto_front(population)
#     return pareto_front

# pop_size = 50
# max_generations = 100

# pareto_front = moead_random_forest(X_train, y_train, X_valid, y_valid, pop_size, max_generations)


TypeError: ignored

In [ ]:
# def moead_random_forest(X_train, y_train, X_valid, y_valid, pop_size, max_generations):
#     population = initialize_population(pop_size)

#     for generation in range(max_generations):
#         weights = generate_weights(pop_size)
#         subproblems = decompose_objectives(population, weights)

#         new_individuals = []
#         for i, subproblem in enumerate(subproblems):
#             parent1, parent2 = select_parents(subproblem, population)
#             child = crossover_and_mutation(parent1, parent2)
#             child_fitness = evaluate_fitness(child['params'], X_train, y_train, X_valid, y_valid)

#             child['fitness']['accuracy'] = child_fitness['accuracy']
#             child['fitness']['diversity'] = child_fitness['diversity']

#             new_individuals.append(child)

#         update_population(population, new_individuals)

#     pareto_front = extract_pareto_front(population)
#     return pareto_front



# pop_size = 50
# max_generations = 100

# # pareto_front = moead_random_forest(X_train, y_train, X_valid, y_valid, pop_size, max_generations)


In [ ]:
# def moead_random_forest(X_train, y_train, X_valid, y_valid, pop_size, max_generations):
#     population = initialize_population(pop_size)

#     for generation in range(max_generations):
#         weights = generate_weights(pop_size)
#         subproblems = decompose_objectives(population, weights)

#         new_individuals = []
#         for i, subproblem in enumerate(subproblems):
#             parent1, parent2 = select_parents(subproblem, population)
#             child = crossover_and_mutation(parent1, parent2)
#             child_fitness = evaluate_fitness(child['params'], X_train, y_train, X_valid, y_valid)

#             child['fitness'] = {
#                 'accuracy': child_fitness['accuracy'],
#                 'diversity': child_fitness['diversity'],
#             }

#             new_individuals.append(child)

#         update_population(population, new_individuals)

#     pareto_front = extract_pareto_front(population)
#     return pareto_front

# pop_size = 50
# max_generations = 100

# pareto_front = moead_random_forest(X_train, y_train, X_valid, y_valid, pop_size, max_generations)


TypeError: ignored

In [ ]:
# import matplotlib.pyplot as plt

# def visualize_pareto_front(pareto_front):
#     fitness_values = [individual['fitness'] for individual in pareto_front]

#     accuracy_values = [fitness['accuracy'] for fitness in fitness_values]
#     diversity_values = [fitness['diversity'] for fitness in fitness_values]

#     plt.scatter(accuracy_values, diversity_values, c='blue', label='Pareto Front')
#     plt.xlabel('Accuracy')
#     plt.ylabel('Diversity')
#     plt.title('Pareto Front')
#     plt.legend()
#     plt.show()

# visualize_pareto_front(pareto_front)